In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

## Load Data

In [3]:
#w q1 train sets
pd_train_w_q1_wo_resamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_w_q1_wo_resamp.csv',header=True).toPandas()

#test sets
pd_test_w_q1 = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_test_w_q1.csv',header=True).toPandas()

In [4]:
# test set dfs to work with
pd_test_w_q1_working = pd_test_w_q1.copy(deep = True)
pd_test_w_q1_working['Second'] = pd_test_w_q1_working['Second'].astype(int)

# Test sets to give sklearn
pd_test_w_q1_4_predict = pd_test_w_q1.drop(['raceId', 'driverId', 'constructorId', 'Second'], axis = 1)

## Model Testing Setup

In [6]:
# Model Test
rf = RandomForestClassifier(n_estimators = 1000, max_depth = 5, max_features = 'sqrt', random_state = 123)
rf_model = rf.fit(X = pd_train_w_q1_wo_resamp.drop(['Second'], axis = 1),
                  y = pd_train_w_q1_wo_resamp[['Second']].values.ravel())
pd_test_w_q1_working['predict_proba'] = rf_model.predict_proba(pd_test_w_q1_4_predict)[:, 1]

In [7]:
# Get max probability per race
max_prob_per_race = pd.DataFrame(pd_test_w_q1_working.groupby(['raceId']).predict_proba.max())
pd_test_w_q1_working = pd_test_w_q1_working\
  .merge(max_prob_per_race, on = 'raceId', how = 'inner', suffixes = ('_driver', '_max'))

In [8]:
# Classify driver with max probability for that race as 1
pd_test_w_q1_working['pred'] = pd_test_w_q1_working['predict_proba_driver']\
  .eq(pd_test_w_q1_working['predict_proba_max']).astype(int)

####  Check if classification procedure is correct

In [10]:
pd_test_w_q1_working['pred'].value_counts()

Out[46]: 0 2864
1 137
Name: pred, dtype: int64

In [11]:
pd_test_w_q1_working['Second'].value_counts()

Out[47]: 0 2864
1 137
Name: Second, dtype: int64

#### Results test

In [13]:
confusion_matrix(pd_test_w_q1_working['Second'], pd_test_w_q1_working['pred'])

Out[48]: array([[2748, 116],
 [ 116, 21]])

In [14]:
precision_score(pd_test_w_q1_working['Second'], pd_test_w_q1_working['pred'])

Out[49]: 0.15328467153284672

In [15]:
# Feature Names
pd_train_w_q1_wo_resamp.drop(['Second'], axis = 1).columns

In [16]:
# Feature importances
rf_model.feature_importances_

Out[50]: array([0.4711558 , 0.04047724, 0.20288236, 0.14996447, 0.05192585,
 0.04020288, 0.04339139])